# Launch LBANN traing jobs
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [1]:
import os
import glob,time
import subprocess as sp
import numpy as np

In [2]:
curr_dir=os.getcwd()
print(curr_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/run_scripts


In [3]:
code_dirs={'128':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/main_code/',
         '512':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/512_sqr/'}

In [22]:
### Default dictionary
img_size=128

dict_pars={'tasks':8, 'epochs':18,
           'seed':36723723, 'nodes':1,'job_name':'spec_test_128_withspec',
           'procs':8, 'time':'4:00:00', 'batchsize':64, 'step_interval':1}

dict_pars['code_dir']=code_dirs[str(img_size)]
print(dict_pars)

{'tasks': 8, 'epochs': 18, 'seed': 36723723, 'nodes': 1, 'job_name': 'spec_test_128_withspec', 'procs': 8, 'time': '4:00:00', 'batchsize': 64, 'step_interval': 1, 'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/main_code/'}


In [23]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes={nodes}
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node=8 
#SBATCH --cpus-per-task=8 
#SBATCH --gpus-per-task=1
#SBATCH --time={time}
#SBATCH --job-name={job_name}

### Initial setup
module purge
module load modules/3.2.11.4 gcc/8.3.0 cuda/10.2.89 mvapich2/2.3.2 cmake/3.14.4 python3/3.7-anaconda-2019.10

module load esslurm
module use /global/cfs/cdirs/m3363/lbann/tom_lbann_install/etc/modulefiles
module load lbann

export MKL_THREADING_LAYER=GNU

#export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/spack.git
export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/tom_spack
$SPACK_ROOT/share/spack/setup-env.sh
export MV2_ENABLE_AFFINITY=0
export MV2_USE_CUDA=1
export IBV_FORK_SAFE=1

### Run the main code
code_dir={code_dir}

python $code_dir/train_exagan.py --seed {seed} -b {batchsize} -stp {step_interval} --nodes {nodes} --procs {procs} --suffix {job_name} --epochs {epochs} 

echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [24]:
# bash_strg

In [25]:
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'

In [26]:
fname='batch_train.sh'
filename=staging_loc+fname
with open (filename,'w') as f:
    f.write(bash_strg)

In [27]:
print(filename)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/batch_train.sh


In [28]:
### Move to staging locations in project space:
os.chdir(staging_loc)

## Submit job to cori GPU

In [29]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node=8 
#SBATCH --cpus-per-task=8 
#SBATCH --gpus-per-task=1
#SBATCH --time=4:00:00
#SBATCH --job-name=spec_test_128_withspec

### Initial setup
module purge
module load modules/3.2.11.4 gcc/8.3.0 cuda/10.2.89 mvapich2/2.3.2 cmake/3.14.4 python3/3.7-anaconda-2019.10

module load esslurm
module use /global/cfs/cdirs/m3363/lbann/tom_lbann_install/etc/modulefiles
module load lbann

export MKL_THREADING_LAYER=GNU

#export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/spack.git
export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/tom_spack
$SPACK_ROOT/share/spack/setup-env.sh
export MV2_ENABLE_AFFINITY=0
export MV2_USE_CUDA=1
export IBV_FORK_SAFE=1

### Run the main code
code_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/main_code/

python $code_dir/train_exagan.py --seed 367237

In [30]:
%%bash -s "$filename" ## Use python variable in bash
module load esslurm
sbatch $1

Submitted batch job 1020786
